<a href="https://colab.research.google.com/github/AkratiSachan23/RAG-System/blob/main/RAG(Q%26A).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval‑Augmented Generation (RAG) Q&A for PS‑4
**National‑Security Focused**  
This notebook will:
1. Install dependencies  
2. Load your PDFs from `./data/`  
3. Extract and clean text  
4. Chunk into passages  
5. Build a FAISS index  
6. Run an interactive Q&A loop with FLAN‑T5  

---



In [ ]:
!pip install faiss-cpu sentence-transformers transformers PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [3]:
# Imports
import os
import fitz                             # PyMuPDF
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [4]:
# Configuration
PDF_FOLDER = "data"                    # place your PDFs here
EMBED_MODEL = "all-MiniLM-L6-v2"
GEN_MODEL   = "google/flan-t5-base"
INDEX_PATH  = "faiss.index"
CHUNK_SIZE  = 500
CHUNK_OVERLAP = 50
TOP_K = 3


In [5]:
# PDF Loader
def load_pdfs(pdf_folder: str):
    docs = {}
    for fname in sorted(os.listdir(pdf_folder)):
        if not fname.lower().endswith(".pdf"): continue
        path = os.path.join(pdf_folder, fname)
        txt_pages = []
        with fitz.open(path) as pdf:
            for page in pdf:
                txt_pages.append(page.get_text())
        docs[fname] = "\n".join(txt_pages)
        print(f"Loaded {fname} ({len(txt_pages)} pages)")
    return docs

docs = load_pdfs(PDF_FOLDER)


Loaded Alert and Advisories _ NCIIPC.pdf (162 pages)
Loaded CERT-In Vulnerability Notes.pdf (2 pages)
Loaded CIWP-2025-0002.pdf (53 pages)
Loaded Comprehensive_Cyber_Security_Audit_Policy_Guidelines.pdf (69 pages)
Loaded National_Cyber_Security_Policy-2013.pdf (15 pages)
Loaded Roles_Responsibilities-CISO.pdf (2 pages)
Loaded SOP-PPP.pdf (25 pages)


In [6]:
# Chunking
splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", " ", ""]
)

chunks = []
for doc_id, text in docs.items():
    parts = splitter.split_text(text)
    for i, chunk in enumerate(parts):
        chunks.append({
            "doc_id": doc_id,
            "chunk_id": i,
            "text": chunk
        })
print(f"Total chunks: {len(chunks)}")


Total chunks: 1200


In [7]:
# Embed & Index
embedder = SentenceTransformer(EMBED_MODEL)

# encode all chunks
texts = [c["text"] for c in chunks]
embeddings = embedder.encode(texts, show_progress_bar=True)
dim = embeddings.shape[1]

# create FAISS index
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings, dtype="float32"))
faiss.write_index(index, INDEX_PATH)
print(f"FAISS index built ({index.ntotal} vectors) and saved to {INDEX_PATH}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


FAISS index built (1200 vectors) and saved to faiss.index


In [8]:
# Helper Classes
class Retriever:
    def __init__(self, index_path=INDEX_PATH):
        self.index = faiss.read_index(index_path)
        self.embedder = embedder

    def retrieve(self, query: str, top_k=TOP_K):
        q_emb = self.embedder.encode([query])
        D, I = self.index.search(np.array(q_emb, dtype="float32"), top_k)
        return [chunks[i] for i in I[0]]

class Generator:
    def __init__(self, model_name=GEN_MODEL):
        self.pipe = pipeline("text2text-generation", model=model_name)

    def answer(self, question: str, contexts: list):
        combined = "\n\n".join(
            f"[{c['doc_id']}#{c['chunk_id']}]: {c['text']}"
            for c in contexts
        )
        prompt = (
            "Use the following document snippets to answer the question and cite each fact:\n\n"
            f"{combined}\n\nQuestion: {question}\nAnswer:"
        )
        out = self.pipe(prompt, max_length=200)[0]["generated_text"]
        return out


In [9]:
# Run Q&A Loop
ret = Retriever()
gen = Generator()

print("RAG Q&A ready! Type your question, or 'exit' to quit.\n")
while True:
    q = input("Q> ").strip()
    if q.lower() in ("exit", "quit"):
        break
    ctx = ret.retrieve(q)
    ans = gen.answer(q, ctx)
    print("\nA:", ans, "\n" + "-"*60 + "\n")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


RAG Q&A ready! Type your question, or 'exit' to quit.

Q> What are current Cyber Security Threats


Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



A: identity theft, phishing, social engineering, hactivism, cyber terrorism, compound threats 
------------------------------------------------------------

Q> exit
